# component

> TODO fill in description

In [ ]:
#| default_exp component

In [ ]:
#| hide
from nbdev.showdoc import *;

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

In [ ]:
#|export
import asyncio
from abc import ABC, abstractmethod
import typing
from typing import Type, Optional, Callable, Any, Union, Tuple
from enum import Enum
import inspect

from fbdev.utils import AttrContainer, create_task_with_exception_handler
from fbdev.packet import Packet
from fbdev.port import PortType, PortSpec, ConfigPortSpec, PortTypeSpec, PortSpecCollection, BasePort, InputPort, ConfigPort, OutputPort, PortCollection

In [ ]:
#|hide
import fbdev

In [ ]:
#|hide
show_doc(fbdev.component.BaseComponent)

---

### BaseComponent

>      BaseComponent ()

*Helper class that provides a standard way to create an ABC using
inheritance.*

In [ ]:
#|export
class BaseComponent(ABC):
    port_specs = PortSpecCollection(
        input=PortTypeSpec(),
        output=PortTypeSpec(),
        config=PortTypeSpec(),
        signal=PortTypeSpec(),
    )
    
    parent_factory = None
    is_factory = False
    is_passive = False # If True, then Component.execute will not be run by its parent Node.

    def __init_subclass__(cls, **kwargs):
        """Prevents any subclass from defining an __init__ that accepts any argument other than 'self'."""
        super().__init_subclass__(**kwargs)
        init_method = cls.__dict__.get('__init__', None)
        if init_method:
            from inspect import signature
            sig = signature(init_method)
            params = tuple([p.name for p in sig.parameters.values()])
            if params != ('self',):
                raise TypeError(f"{cls.__name__}.__init__() must only accept 'self' as an argument.")
    
    def __init__(self):
        self.ports = PortCollection(self.port_specs, self)
        self.config = AttrContainer({}, obj_name="Component.config")
        for config_port_name, config_port_spec in self.port_specs.config.items():
            if config_port_spec.has_default:
                self.set_config(config_port_name, config_port_spec.default)
                    
    async def set_config(self, name:str, packet:Packet):
        value = await packet.consume()
        if self.port_specs.config[name].has_dtype and type(value) != self.port_specs.config[name].dtype:
            raise TypeError(f"Config value {value} is not of type {self.port_specs.config[name].dtype}, in config port {name}.")
        if self.port_specs.config[name].has_data_validator and not self.port_specs.config[name].data_validator(value):
            raise ValueError(f"Config value {value} is not valid for config port {name}, in config port {name}.")
        if name not in self.port_specs.config.keys():
            raise ValueError(f"Config port {name} is not a valid config port for component {self.__class__.__name__}.")
        self.config._set(name, value)
        
    def check_configured(self) -> bool:
        configured = True
        for config_port_name, config_port_spec in self.port_specs.config.items():
           if not config_port_name in self.config and not config_port_spec.is_optional:
               configured = False
               break
        return configured
    
    @classmethod
    def is_from_factory(cls):
        return cls.parent_factory is not None
    
    @abstractmethod
    async def execute(self):
        if self.is_passive:
            raise RuntimeError("Component is passive, so it cannot be executed.")
    
    async def start_background_tasks(self):
        pass
    
    async def stop_background_tasks(self):
        pass
    
    async def destroy(self):
        await self.stop_background_tasks()
    
    def __del__(self):
        loop = asyncio.get_event_loop()
        if loop.is_running(): create_task_with_exception_handler(self.destroy())
        else: loop.run_until_complete(self.destroy())

In [ ]:
class MyComponent(BaseComponent):
    port_specs = PortSpecCollection(
        input=PortTypeSpec(
            in1=PortSpec(dtype=str, data_validator=lambda x: x.endswith("world")),
        ),
        output=PortTypeSpec(
            out1=PortSpec(dtype=str),
        ),
        config=PortTypeSpec(
            greeting=ConfigPortSpec(dtype=str),
            append=ConfigPortSpec(dtype=str),
        ),
        signal=PortTypeSpec(),
    )
    
    async def execute(self):
        packet = await self.ports.input.in1.receive()
        packet_payload = await packet.consume()
        await self.ports.output.out1.put(Packet(f"{self.config.greeting} {packet_payload}{self.config.append}"))
        
comp_process = MyComponent()

async def packet_sender():
    await comp_process.ports.input.in1._requesting_packet.wait()
    comp_process.ports.input.in1._load_packet(Packet("world"))
async def packet_receiver():
    await comp_process.ports.output.out1._ready_to_unload.wait()
    packet = comp_process.ports.output.out1._unload_packet()
    packet_payload = await packet.consume()
    print(packet_payload)

await comp_process.set_config("greeting", Packet("Hello"))
await comp_process.set_config("append", Packet("!"))

await asyncio.gather(
    asyncio.create_task(packet_sender()),
    asyncio.create_task(comp_process.execute()),
    asyncio.create_task(packet_receiver()),
)

await comp_process.set_config("greeting", Packet("Goodbye"))
await comp_process.set_config("append", Packet("?"))

await asyncio.gather(
    asyncio.create_task(packet_sender()),
    asyncio.create_task(comp_process.execute()),
    asyncio.create_task(packet_receiver()),
);

Hello world!
Goodbye world?


In [ ]:
#|hide
show_doc(fbdev.component.ComponentFactory)

---

### ComponentFactory

>      ComponentFactory ()

*Helper class that provides a standard way to create an ABC using
inheritance.*

In [ ]:
#|export
class ComponentFactory(BaseComponent):
    is_factory = True
    
    def __init_subclass__(cls, **kwargs):
        """Overloads `BaseComponent.__init_subclass__`, to allow ComponentFactory subclasses to have constructors with arguments."""
        pass

    @classmethod
    def _create_component_class(cls, component_name=None, class_attrs={}, init_args=[], init_kwargs={}):
        if component_name is None:
            if cls.__name__.endswith("Factory"):
                component_name = cls.__name__[:-len("Factory")]
            else:
                component_name = cls.__name__
        return type(component_name, (cls,), {
            '__init__': lambda self: cls.__init__(self, *init_args, **init_kwargs),
            'parent_factory' : cls,
            'is_factory' : False,
            **class_attrs
        })

    @classmethod
    def get_component(cls, *init_args, **init_kwargs):
        """Creates a new instance of the component class, with the given arguments.
        Overload to modify the behaviour (for example, to allow modification of `port_spec`)
        """
        return cls._create_component_class()

In [ ]:
class SummationComponentFactory(ComponentFactory):
    def __init__(self, message):
        super().__init__()
        self._message = message
        
    @classmethod
    def get_component(cls, num_summands:int, message:str):
        port_specs = PortSpecCollection(
            input=PortTypeSpec(**{f'summand_{i}' : PortSpec() for i in range(num_summands)}),
            output=PortTypeSpec(out=PortSpec(dtype=str)),
            config=PortTypeSpec(),
            signal=PortTypeSpec(),
        )
        return cls._create_component_class(class_attrs={'port_specs' : port_specs}, init_args=[message])
    
    async def execute(self):
        sum = 0
        for port in self.ports.input.values():
            packet = await port.receive()
            sum += await packet.consume()
        await self.ports.output.out.put(Packet(f"{self._message} {sum}"))
    
num_summands = 5
SummationComponent = SummationComponentFactory.get_component(num_summands, "The sum is: ")
comp_process = SummationComponent()

async def packet_sender():
    for i, port in enumerate(comp_process.ports.input.values()):
        await port._requesting_packet.wait()
        port._load_packet(Packet(i))
async def packet_receiver():
    await comp_process.ports.output.out._ready_to_unload.wait()
    packet = comp_process.ports.output.out._unload_packet()
    packet_payload = await packet.consume()
    print(packet_payload)

await asyncio.gather(
    asyncio.create_task(packet_sender()),
    asyncio.create_task(comp_process.execute()),
    asyncio.create_task(packet_receiver()),
);

The sum is:  10


In [ ]:
#|hide
show_doc(fbdev.component.DummyComponentFactory)

---

### DummyComponentFactory

>      DummyComponentFactory ()

*Helper class that provides a standard way to create an ABC using
inheritance.*

In [ ]:
#|export
class DummyComponentFactory(ComponentFactory):
    @classmethod
    def get_component(cls, port_specs:PortSpecCollection, component_name=None):
        return cls._create_component_class(component_name=component_name, class_attrs={'port_specs':port_specs})
    
    async def execute(self):
        for port_name, port in self.ports.input.items():
            packet = await port.receive()
            packet_payload = await packet.consume()
            
        for port_name, port in self.ports.output.items():
            if port.dtype: data = port.dtype()
            else: data = None
            packet = Packet(data)
            await port.put(packet)

In [ ]:
dummy_component = DummyComponentFactory.get_component(PortSpecCollection(
    input=PortTypeSpec(in1=PortSpec(dtype=int)),
    output=PortTypeSpec(out1=PortSpec(dtype=int))
))

dummy_component.port_specs

PortType.INPUT:
  in1:int
PortType.OUTPUT:
  out1:int
PortType.CONFIG:
PortType.SIGNAL:

In [ ]:
#|hide
show_doc(fbdev.component.FunctionComponentFactory)

---

### FunctionComponentFactory

>      FunctionComponentFactory (func)

*Helper class that provides a standard way to create an ABC using
inheritance.*

In [ ]:
#|export
class FunctionComponentFactory(ComponentFactory):
    def __init__(self, func):
        self._func = func
        super().__init__()
        
    @classmethod
    def get_component(cls, func, component_name=None):
        if component_name is None: component_name = func.__name__
        
        input_ports = {}
        config_ports = {}
        
        # Input and config ports
        signature = inspect.signature(func)
        for param in signature.parameters.values():
            port_name = param.name
            if isinstance(param.annotation, PortSpec):
                port_spec = param.annotation
                port_spec.name = port_name
                port_spec._port_type = PortType.CONFIG if type(param.annotation) == ConfigPortSpec else PortType.INPUT
                if port_spec.port_type == PortType.CONFIG:
                    if port_spec.has_default and param.default != inspect.Parameter.empty:
                        raise ValueError(f"Config port {port_name} is supplied multiple default values.")
            elif type(param.annotation) == type:
                if param.default != inspect.Parameter.empty:
                    port_spec = ConfigPortSpec(port_name, dtype=param.annotation, default=param.default)
                else:
                    port_spec = PortSpec(port_name, port_type=PortType.INPUT, dtype=param.annotation)
            elif param.annotation == inspect.Parameter.empty:
                if param.default != inspect.Parameter.empty:
                    port_spec = ConfigPortSpec(port_name, default=param.default)
                else:
                    port_spec = PortSpec(port_name, port_type=PortType.INPUT)
            else:
                raise ValueError(f"Unsupported annotation {param.annotation}")
            
            if port_spec.port_type == PortType.CONFIG: config_ports[port_name] = port_spec
            elif port_spec.port_type == PortType.INPUT: input_ports[port_name] = port_spec
        
        # Output ports
        if type(signature.return_annotation) == PortTypeSpec:
            output_ports = signature.return_annotation
        elif typing.get_origin(signature.return_annotation) == tuple:
            output_ports = PortTypeSpec(**{
                f'out{i}' : PortSpec(port_type=PortType.OUTPUT, dtype=t)
                for i, t in enumerate(typing.get_args(signature.return_annotation))
            })
        elif signature.return_annotation == inspect.Parameter.empty:
            output_ports = PortTypeSpec(out=PortSpec(port_type=PortType.OUTPUT))
        elif type(signature.return_annotation) == type:
            output_ports = PortTypeSpec(out=PortSpec(port_type=PortType.OUTPUT, dtype=signature.return_annotation))
        elif signature.return_annotation == None:
            output_ports = PortTypeSpec()
        else:
            raise ValueError(f"Unsupported return annotation {signature.return_annotation}")
        
        port_specs = PortSpecCollection(
            input=PortTypeSpec(**input_ports),
            config=PortTypeSpec(**config_ports),
            output=output_ports
        )
        
        return cls._create_component_class(component_name=component_name, class_attrs={'port_specs':port_specs}, init_args=[func])
    
    async def execute(self):
        await super().execute()
        kwargs = {}
        for port_name, port in self.ports.input.items():
            packet = await port.receive()
            packet_payload = await packet.consume()
            kwargs[port_name] = packet_payload
        for config_name, config_value in self.ports.config.items():
            kwargs[config_name] = config_value
            
        output = self._func(**kwargs)
        
        if len(self.ports.output) == 1:
            await self.ports.output.out.put(Packet(output))
        elif len(self.ports.output) > 1:
            if type(output) == tuple or type(output) == list:
                for output_i, port_i in zip(output, self.ports.output.values()):
                    await port_i.put(Packet(output_i))
            elif type(output) == dict:
                for output_key, output_val in output.items():
                    await self.ports.output[output_key].put(Packet(output_val))
            else:
                raise RuntimeError(f"Unsupported output type {type(output)}.")
    
    

In [ ]:
#|export
def func_component(name=None):
    def decorator(func):
        return FunctionComponentFactory.get_component(func, name)
    return decorator

In [ ]:
@func_component()
def add_one(a:int) -> int:
    print("add_one")
    return a+1